# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.datasets import  fetch_california_housing

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [4]:
spaceship["Cabin"].nunique
spaceship["Cabin"] = spaceship["Cabin"].str.split('/').str[0]
spaceship.drop(["PassengerId", "Name"], axis =1, inplace = True)
spaceship = spaceship.dropna()
spaceship.isnull().sum()

HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

In [8]:
categorical_data = spaceship.select_dtypes("object")
categorical_data

# get dummies for categorical data
data_dummies = pd.get_dummies(categorical_data, drop_first=True)
data_dummies = data_dummies.astype(int)

# numerical features
numerical_data = spaceship.select_dtypes("number")

features = pd.concat([numerical_data, data_dummies], axis=1) # combine numerical and categorical features
target = spaceship["Transported"].astype(int) # define the target

**Perform Train Test Split**

In [11]:
# performing train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

# normalize numerical features
normalizer = MinMaxScaler()
normalizer.fit(X_train)

#scale numerical features
scaler = StandardScaler()
scaler.fit(X_train)


X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

X_train_norm_df = pd.DataFrame(X_train_norm, columns = X_train.columns, index=X_train.index)
X_test_norm_df = pd.DataFrame(X_test_norm, columns = X_test.columns, index=X_test.index)

X_train_standarized_np = scaler.transform(X_train)
X_test_standarized_np = scaler.transform(X_test)

X_train_standarized_df = pd.DataFrame(X_train_standarized_np, columns = X_train.columns, index=X_train.index)
X_test_standarized_df  = pd.DataFrame(X_test_standarized_np, columns = X_test.columns, index=X_test.index)

In [13]:
X_test_norm_df.head()
X_train_standarized_df.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
6201,0.633057,-0.353053,-0.168253,-0.316237,0.903105,3.331720,1.717450,-0.510317,-0.731759,-0.318377,-0.313423,-0.244405,2.931548,-0.696645,-0.652264,-0.019229,-0.325717,-1.507794,-0.158129
6106,0.633057,-0.353053,-0.290009,-0.316237,-0.280524,-0.271427,-0.582259,-0.510317,1.366570,-0.318377,-0.313423,-0.244405,-0.341117,-0.696645,1.533122,-0.019229,3.070150,-1.507794,-0.158129
7095,0.702161,-0.353053,-0.290009,-0.316237,-0.280524,-0.271427,-0.582259,1.959564,1.366570,-0.318377,-0.313423,-0.244405,-0.341117,1.435451,-0.652264,-0.019229,-0.325717,-1.507794,-0.158129
3060,-1.992884,-0.353053,-0.290009,-0.316237,-0.280524,-0.271427,-0.582259,-0.510317,-0.731759,-0.318377,-0.313423,-0.244405,-0.341117,-0.696645,1.533122,-0.019229,-0.325717,-1.507794,-0.158129
4101,0.909472,1.106502,-0.290009,-0.314455,-0.280524,-0.271427,-0.582259,1.959564,-0.731759,-0.318377,-0.313423,-0.244405,2.931548,-0.696645,-0.652264,-0.019229,-0.325717,0.663221,-0.158129


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [17]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

In [19]:
bagging_reg.fit(X_train_norm, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

In [56]:
pred = bagging_reg.predict(X_test_norm)

print(f"MAE, {mean_absolute_error(pred, y_test): .2f}")
print(f"MSE, {mean_squared_error(pred, y_test): .2f}")
print(f"RMSE, {root_mean_squared_error(pred, y_test): .2f}")
print(f"R2 score, {bagging_reg.score(X_test_norm, y_test): .2f}")

MAE,  0.27
MSE,  0.13
RMSE,  0.36
R2 score,  0.47


- Random Forests

In [24]:
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=20)

In [26]:
forest.fit(X_train_norm, y_train)

RandomForestRegressor(max_depth=20)

In [58]:
pred = forest.predict(X_test_norm)

print(f"MAE, {mean_absolute_error(pred, y_test): .2f}")
print(f"MSE, {mean_squared_error(pred, y_test): .2f}")
print(f"RMSE, {root_mean_squared_error(pred, y_test): .2f}")
print(f"R2 score, {forest.score(X_test_norm, y_test): .2f}")

MAE,  0.26
MSE,  0.14
RMSE,  0.38
R2 score,  0.43


- Gradient Boosting

In [29]:
gb_reg = GradientBoostingRegressor(max_depth=20,
                                   n_estimators=100)

In [30]:
gb_reg.fit(X_train_norm, y_train)

GradientBoostingRegressor(max_depth=20)

In [62]:
pred = gb_reg.predict(X_test_norm)

print(f"MAE, {mean_absolute_error(pred, y_test): .2f}")
print(f"MSE, {mean_squared_error(pred, y_test): .2f}")
print(f"RMSE, {root_mean_squared_error(pred, y_test): .2f}")
print(f"R2 score, {gb_reg.score(X_test_norm, y_test): .2f}")

MAE,  0.27
MSE,  0.18
RMSE,  0.43
R2 score,  0.26


- Adaptive Boosting

In [43]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)

In [46]:
ada_reg.fit(X_train_norm, y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                  n_estimators=100)

In [60]:
pred = ada_reg.predict(X_test_norm)

print(f"MAE, {mean_absolute_error(pred, y_test): .2f}")
print(f"MSE, {mean_squared_error(pred, y_test): .2f}")
print(f"RMSE, {root_mean_squared_error(pred, y_test): .2f}")
print(f"R2 score, {ada_reg.score(X_test_norm, y_test): .2f}")

MAE,  0.23
MSE,  0.16
RMSE,  0.40
R2 score,  0.36


Which model is the best and why?

In [ ]:
#comment here